# Detecção automátia de *DeepFakes*
## Uma abordagem utilizando redes neurais
---

### 1. Visão Geral do Problema

#### O que afinal é um DeepFake?
- A grosso modo é uma técnica utilizada para produzir imagens e **sons** humanos realistas utilizando Deep Learning (origem do 'Deep' no nome).

<img src="./Imagens/deepfake-example.gif" width="500" height="600"/>

#### Acessibilidade x Resultados
O grande problema que está associado ao DeepFake hoje é a relação acessibilidade x resultados. Antigamente, para se produzir um Deepfake, era necessário equipamento/software de ponta, muito talento artístico para tal e processamento computacional ainda muito inacessível. Eram poucas pessoas. O que foi alterado nessa equação nos dias de hoje?

- ~TALENTO~

Hoje temos a ascenção da inteligência artificial na forma de GANs (Generative Adversarial Network). A compactação e eventual produção em massa dessa arquitetura aplicada à codificação e decodificação de rostos removeu da equação a necessidade de um elevado talento humano para a produção de Deepfakes realistas. GANs são entidades extremamente poderosas capazes de produzir imagens extremamente realistas e que muitas vezes enganam a visão humana. Tudo que elas precisam é dados para treinamento na forma de duas entradas: o rosto/som que se deseja ser trocado e o rosto/som original que deve ser substituído.

- ~DIFÍCIL ACESSIBILIDADE~

Hoje, com essa ascenção, não mais é necessário um estúdio ou equipamento de ponta para a produção de Deepfakes. A fácil compressão dessas arquiteturas de rede torna possível sua produção em APPs de smartphone e softwares simples de computadores pessoais. Coisas que todos nós temos acesso hoje.

Apps chineses como ZAO são capazes de produzir deepfakes realistas em questão de segundos (ainda só funciona com vídeos da própria plataforma).

<img src="./Imagens/zao_ex1.gif"  width="800" height="100"/>

<img src="./Imagens/zao_ex2.gif"  width="800" height="100"/>

Um dos softwares mais famosos para isso hoje é o `DeepFaceLab`, que requer apenas uma instalação simples de windows.

<img src="./Imagens/DeepFaceLab.PNG"/>

- ~HARDWARE DE PONTA~

Como essas arquiteturas muitas vezes podem conter milhões de parâmetros que basicamente se resumem no final a serem utilizados em matrizes que serão multiplicadas (contas simples, milhões delas), a capacidade de uma máquina paralelizar esse treinamento da rede se torna extremamente necessário. Hoje, com o forte mercado de placas gráficas para jogos constantemente em ascenção e o interesse em alta a todo momemnto, temos acesso à placas gráficas extremamente eficientes no treinamento destas redes por um preço bem acessível.

#### Qual o problema?
Como agora temos uma tecnologia extremamente poderosa na produção de deepfakes e extremamente acessível, temos uma quantidade significativa de pessoas utilizando essa tecnologia para fins maliciosos. Não é incomum achar hoje vídeos de pessoas como Mark Zuckerberg falando coisas que nunca falaria, quando na verdade aquilo é um deepfake realizado por um entusiasta qualquer no computador de casa.

<img src="./Imagens/mzdf.gif"/>

Isso traz o problema do até quanto será possível obter uma informação virtual e ter certeza que aquela informação é verídica? Hoje já batemos nessa barreira.

#### Exemplo de facilidade

### 2. Objetivos Principais

- Queremos ser capazes de detectar se algum vídeo em questão contém a presença de DeepFakes faciais (seja de 1 rosto ou vários rostos) utilizando modelos pré treinados no ImageNET e disponíveis no framework Pytorch através da biblioteca FastAI.
- Explorar componentes de luminância para ajudar no treinamento das redes.
- Observar o quão benéfico é ou não é utilizar essa técnica em relação a técnicas e soluções já propostas na área.
- Observar caminhos para a colocação em produção.

### 3. Base de Dados

Sabemos que redes neurais são mapeadores universais de A -> B:

- Animal -> É um cachorro? (classificação)
- Dados -> Ações vão subir? (classificação)
- Paciente -> Dosagem ideal (regressão)

Na abordagem que será adotada em questão:
- Vídeo -> Contém deepfake? (classificação)

Estamos lidando a primeiro momento com um problema de classificação supervisionado. Precisamos de dados:

<img src="./Imagens/kaggledfdc.PNG"/>

Dataset extenso de aproximadamente `500GB` organizado em `50 pastas`, cada uma contendo `milhares` de vídeos (1000 ~ 3000 vídeo por pasta).

- A proporção de vídeos FAKE para vídeos REAL é de 5:1 por pasta, aproximadamente.
- As classes de cada vídeos estão organizadas por um arquivo `metadata.JSON` presente em cada uma das postas, controlando os vídeos daquela pasta em particular.


### 4. Metodologia

A abordagem aqui empregada consiste em X etapas:

**1.** A primeira parte será extrair os rostos de todos os vídeos de todas as pastas utilizando alguma estratégia e organizá-los em 2 novas pastas: `FAKE` e `REAL`.

**2.** Utilizando essas duas novas pastas FAKE e REAL, separar as imagens em `treino`, `validação` e `teste` utilizando `Validação Cruzada` 5 folds.

**3.** Carregar as redes pré treinadas disponíveis no Pytorch (Resnets 18 a 50, EfficientNet).

**4.** Define-se as características de treinamento e os hiperparâmetros:

- Tamanho da imagem.
- Tamanho do batch.
- Função de custo que queremos minimizar e seus hiperparâmetros.
- Métrica para o treinamento e a validação.
- Otimizador.
- LR scheduler.
- Dropout.
- Momentum.
- Política de aprendizagem.
- Callbacks (interromper caso a métrica não melhores / salvar o melhor modelo).

**5.** Treinar cada modelo com os hiperparâmetros definidos sob todos os folds e observar a métrica em questão (exemplo: acurácia), tendo então uma ideia da performance esperada para essa aborgadem.

**6.** Encontrando um bom balanço de hiperparâmetros e noção de performance para cada modelo, treinar cada modelo novamente do dataset completo e verificar sua performance final no conjunto de `teste`.

**7.** Encontrando um bom balanço de hiperparâmetros e noção de performance para cada modelo, treinar cada modelo novamente do dataset completo e verificar sua performance final no conjunto de `teste`.

**8.** Definimos uma estratégia para a classificação final individual de cada vídeo e aplicamos os modelos finais gerados x frames a x frames no dataset de vídeos.

**9.** Analisamos os resultados para cada modelo.